<a href="https://colab.research.google.com/github/taksaphat/gas_emission_model/blob/main/Model_final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all necessary modules

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
import pandas as pd
import seaborn as sns  # to plot images and statical version of data
import numpy as np
import matplotlib.pyplot as plt
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
degree=3
from sklearn.metrics import r2_score
from sklearn import metrics
import csv

get dataset from google drive

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
data = drive.CreateFile({'id':"1toEuurwskgECiHDSQhOYG6swQRfoTFzD"})  
data.GetContentFile('emission data_fix.csv')  

In [4]:
dataset = pd.read_csv('emission data_fix.csv')
dataset.head()

,Country,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,9.191295e+07,1.003652e+08,1.125912e+08,1.233332e+08,1.333337e+08,1.431228e+08,1.532303e+08,1.654882e+08,1.785029e+08,NaN
1,Africa,0,0,0,0,0,0,0,0,0,...,3.301904e+10,3.421283e+10,3.541120e+10,3.664504e+10,3.789569e+10,3.918617e+10,4.047518e+10,4.178583e+10,4.311757e+10,NaN
2,Albania,0,0,0,0,0,0,0,0,0,...,2.331696e+08,2.377643e+08,2.430001e+08,2.479062e+08,2.529662e+08,2.586784e+08,2.646261e+08,2.708990e+08,2.772782e+08,NaN
3,Algeria,0,0,0,0,0,0,0,0,0,...,3.015005e+09,3.132819e+09,3.252626e+09,3.380736e+09,3.513171e+09,3.656348e+09,3.806940e+09,3.957319e+09,4.107870e+09,NaN
4,Americas (other),0,0,0,0,0,0,0,0,0,...,7.961787e+10,8.187178e+10,8.416656e+10,8.654197e+10,8.894874e+10,9.139192e+10,9.382747e+10,9.624253e+10,9.864116e+10,NaN


In [5]:
fieldnames = ['year2018', 'year2019','year2020','year2021','year2022','year2023','year2024','year2025','year2026','year2027','year2028','year2029','year2030','year2031','year2032','accuracy','name']
with open('countries_prediction.csv', 'w', encoding='UTF8') as f:
  writer = csv.writer(f)

  # write the header
  writer.writerow(fieldnames)

  for i in range(229):
    pick = i
    xdata = np.arange(1958,2018)
    ydata = dataset.loc[pick, ['1958', '1959',
          '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
          '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981',
          '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
          '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
          '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
          '2015', '2016', '2017']].values
    # เลือกประเทศที่ข้อมูลไม่ครบออก
    if (ydata[0] == 0):
      continue
    # split, train and fit data
    x = xdata
    y = ydata
    x = xdata.reshape(-1,1)
    #split 80/20
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
    lm=make_pipeline(PolynomialFeatures(degree),LinearRegression())
    lm.fit(X_train,y_train)
    # test accuracy of trained data (R2_Score)
    predictions = lm.predict(X_test)
    score_pred = lm.predict(X_test)
    score_real = y_test
    np.ravel(lm.predict(X_test), order='C')
    np.ravel(y_test, order='C')
    r2_score(score_real,score_pred)
    r2print = r2_score(score_real,score_pred)
    # prediction
    ghgGrow = np.arange(2018, 2032+1)
    ghgGrow = ghgGrow.reshape(-1,1)
    ghgGrow_pred = lm.predict(ghgGrow)
    strCountry = str(dataset.loc[pick,['Country']].values)
    ghgGrow_w = ghgGrow_pred.tolist()
    ghgGrow_w.append(r2print)
    ghgGrow_w.append(strCountry)
    # write result in csv
    writer.writerow(ghgGrow_w)